<a href="https://colab.research.google.com/github/ykitaguchi77/GO_AI_project/blob/main/make_super_extend_dataset_for_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Create extended dataset for YOLOv5 implementation**

#Olympia
・For_validation ：treated 169枚、control 169枚 ここからテスト用に残すもの→treated 69枚、control 69枚 controlのうち100枚→train/valに回す

・Treated 640px：treated 561枚 テスト用に残した69枚以外もの全て（grav:492枚）をtrain/valに回す

#Handai
・GO_extended_dataset

control: 1886枚

treatable: 1879枚

#合計：
treatable: 1879 + 492

control: 1886 + 100

testset: Olympia treated 69, control 69

##既存のYOLOでbounding boxを作成 --> classは既知のもので。Glaucomaのrepositoryを参考にしてYOLO datasetを作成。

In [3]:
import pandas as pd
import os
import glob
import shutil
import glob
import random
from PIL import Image
import time
%matplotlib inline

#サポートパッチのインポート
from google.colab.patches import cv2_imshow
import cv2


# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
dst_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO"

if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)
os.makedirs(dst_dir)
os.makedirs(f"{dst_dir}/train_val")
os.makedirs(f"{dst_dir}/train_val/grav")
os.makedirs(f"{dst_dir}/train_val/cont")
os.makedirs(f"{dst_dir}/test")
os.makedirs(f"{dst_dir}/test/grav")
os.makedirs(f"{dst_dir}/test/cont")


